In [138]:
import pandas as pd

In [139]:
orders_df = pd.read_csv('../staging/orders/orders.csv')
order_items_df = pd.read_csv('../staging/order_items/order_items.csv')
exchange_df = pd.read_csv('../staging/exchange/exchange.csv')
customers_df = pd.read_csv('../staging/customers/customers.csv')
stores_df = pd.read_csv('../staging/stores/stores.csv')
brands_df = pd.read_csv('../staging/brands/brands.csv')
categories_df = pd.read_csv('../staging/categories/categories.csv')
products_df = pd.read_csv('../staging/products/products.csv')
staffs_df = pd.read_csv('../staging/staffs/staffs.csv')
stocks_df = pd.read_csv('../staging/stocks/stocks.csv')

In [140]:
order_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4639 entries, 0 to 4638
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      4639 non-null   int64  
 1   item_id       4639 non-null   float64
 2   product_id    4639 non-null   int64  
 3   quantity      4639 non-null   int64  
 4   list_price    4639 non-null   float64
 5   discount      4639 non-null   float64
 6   extract_date  4639 non-null   object 
 7   source        4639 non-null   object 
dtypes: float64(3), int64(3), object(2)
memory usage: 290.1+ KB


In [141]:
exchange_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     1 non-null      object 
 1   EGP           1 non-null      float64
 2   extract_date  1 non-null      object 
 3   source        1 non-null      object 
dtypes: float64(1), object(3)
memory usage: 160.0+ bytes


In [142]:
order_items_df['New'] = 1
exchange_df['New'] = 1

In [143]:
merge_df = pd.merge(order_items_df ,exchange_df , on = 'New', how= 'left')

In [144]:
order_items_df['list_price_egp'] = merge_df['list_price'] * merge_df['EGP']
order_items_df.drop(columns=['New'], inplace=True)

In [145]:
order_items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4639 entries, 0 to 4638
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        4639 non-null   int64  
 1   item_id         4639 non-null   float64
 2   product_id      4639 non-null   int64  
 3   quantity        4639 non-null   int64  
 4   list_price      4639 non-null   float64
 5   discount        4639 non-null   float64
 6   extract_date    4639 non-null   object 
 7   source          4639 non-null   object 
 8   list_price_egp  4639 non-null   float64
dtypes: float64(4), int64(3), object(2)
memory usage: 326.3+ KB


In [146]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       1615 non-null   int64 
 1   customer_id    1615 non-null   int64 
 2   order_status   1615 non-null   int64 
 3   order_date     1615 non-null   object
 4   required_date  1615 non-null   object
 5   shipped_date   1615 non-null   object
 6   store_id       1615 non-null   int64 
 7   staff_id       1615 non-null   int64 
 8   extract_date   1615 non-null   object
 9   source         1615 non-null   object
dtypes: int64(5), object(5)
memory usage: 126.3+ KB


In [147]:
orders_df['required_date'] = pd.to_datetime(orders_df['required_date'])
orders_df['shipped_date'] = pd.to_datetime(orders_df['shipped_date'])
orders_df['late_delivery'] = orders_df['required_date'] < orders_df['shipped_date']

In [148]:
orders_df['latency days'] = (orders_df['shipped_date'] - orders_df['required_date'])
orders_df.loc[orders_df['latency days'] < pd.Timedelta(days=1), 'latency days'] = pd.NaT

In [149]:
orders_df['weekend'] = (orders_df['shipped_date'].dt.dayofweek == 6) | (orders_df['shipped_date'].dt.dayofweek == 0)

In [150]:
orders_df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_date,source,late_delivery,latency days,weekend
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-28 18:15:34.177976,postgre,False,NaT,True
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-28 18:15:34.177976,postgre,False,NaT,True
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-28 18:15:34.177976,postgre,False,NaT,True
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-28 18:15:34.177976,postgre,True,1 days,False
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-28 18:15:34.177976,postgre,False,NaT,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,1611,6,3,2018-09-06,2018-09-06,2018-04-02,2,7,2024-04-28 18:15:34.177976,postgre,False,NaT,True
1611,1612,3,3,2018-10-21,2018-10-21,2018-04-02,1,3,2024-04-28 18:15:34.177976,postgre,False,NaT,True
1612,1613,1,3,2018-11-18,2018-11-18,2018-04-02,2,6,2024-04-28 18:15:34.177976,postgre,False,NaT,True
1613,1614,135,3,2018-11-28,2018-11-28,2018-04-02,3,8,2024-04-28 18:15:34.177976,postgre,False,NaT,True


In [151]:
customers_df['local'] = customers_df['city'].isin(stores_df['city'])

In [152]:
customers_df

,customer_id,first_name,last_name,phone,email,street,city,state,zip_code,extract_date,source,local
0,1,Debra,Burks,not-available,debra.burks@yahoo.com,9273 Thorne Ave.,Orchard Park,NY,14127,2024-04-28 18:21:26.181409,datalake,False
1,2,Kasha,Todd,not-available,kasha.todd@yahoo.com,910 Vine Street,Campbell,CA,95008,2024-04-28 18:21:26.181409,datalake,False
2,3,Tameka,Fisher,not-available,tameka.fisher@aol.com,769C Honey Creek St.,Redondo Beach,CA,90278,2024-04-28 18:21:26.181409,datalake,False
3,4,Daryl,Spence,not-available,daryl.spence@aol.com,988 Pearl Lane,Uniondale,NY,11553,2024-04-28 18:21:26.181409,datalake,False
4,5,Charolette,Rice,(916) 381-6003,charolette.rice@msn.com,107 River Dr.,Sacramento,CA,95820,2024-04-28 18:21:26.181409,datalake,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1440,1441,Jamaal,Morrison,not-available,jamaal.morrison@msn.com,796 SE. Nut Swamp St.,Staten Island,NY,10301,2024-04-28 18:21:26.181409,datalake,False
1441,1442,Cassie,Cline,not-available,cassie.cline@gmail.com,947 Lafayette Drive,Brooklyn,NY,11201,2024-04-28 18:21:26.181409,datalake,False
1442,1443,Lezlie,Lamb,not-available,lezlie.lamb@gmail.com,401 Brandywine Street,Central Islip,NY,11722,2024-04-28 18:21:26.181409,datalake,False
1443,1444,Ivette,Estes,not-available,ivette.estes@gmail.com,88 N. Canterbury Ave.,Canandaigua,NY,14424,2024-04-28 18:21:26.181409,datalake,False


In [153]:
status_mapping = {1: 'Pending', 2: 'Processing', 3: 'Rejected', 4: 'Completed'}

status_lookup_df = pd.DataFrame(status_mapping.items(), columns=['order_status', 'status_description'])

In [154]:
orders_with_status = pd.merge(orders_df, status_lookup_df, on='order_status', how='left')

orders_df['order_status_desc'] = orders_with_status['status_description']
orders_df

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extract_date,source,late_delivery,latency days,weekend,order_status_desc
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Completed
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Completed
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Completed
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-28 18:15:34.177976,postgre,True,1 days,False,Completed
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-28 18:15:34.177976,postgre,False,NaT,False,Completed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,1611,6,3,2018-09-06,2018-09-06,2018-04-02,2,7,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Rejected
1611,1612,3,3,2018-10-21,2018-10-21,2018-04-02,1,3,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Rejected
1612,1613,1,3,2018-11-18,2018-11-18,2018-04-02,2,6,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Rejected
1613,1614,135,3,2018-11-28,2018-11-28,2018-04-02,3,8,2024-04-28 18:15:34.177976,postgre,False,NaT,True,Rejected


In [156]:
brands_df.to_csv('../staging_2/brands/brands.csv', index = 0)
exchange_df.to_csv('../staging_2/exchange/exchange.csv', index = 0)
categories_df.to_csv('../staging_2/categories/categories.csv', index = 0)
orders_df.to_csv('../staging_2/orders/orders.csv', index = 0)
order_items_df.to_csv('../staging_2/order_items/order_items.csv', index = 0)
customers_df.to_csv('../staging_2/customers/customers.csv', index = 0)
products_df.to_csv('../staging_2/products/products.csv', index = 0)
staffs_df.to_csv('../staging_2/staffs/staffs.csv', index = 0)
stocks_df.to_csv('../staging_2/stocks/stocks.csv', index = 0)
stores_df.to_csv('../staging_2/stores/stores.csv', index = 0)
status_lookup_df.to_csv('../staging_2/order_status_lookup/order_status_lookup.csv', index = 0)